# Rank reversals

This tutorial provides an overview of the use of the scikit-criteria ranking comparison tools.

## Context and motivation

Multi-criteria decision methods may present irregularities in their rankings that compromise the reliability of their results.
Depending on how the alternatives are defined and evaluated in a problem, rank reversals are classified in five types.
To evaluate the robustness of these methods, Wang & Triantaphyllou (2008) proposed three test criteria that cover all five kinds of rank reversals.

1. **Rank invariance**: The rank of an optimal alternative must remain invariant when a sub-optimal alternative is worsened.
2. **Transitivity**: when a decision problem is partitioned in smaller sub-problems, the relative transitivity of alternatives must be retained.
2. **Recomposition consistency**: a new ranking constructed from a set of smaller sub-problems must be equivalent to the ranking obtained from running the original method.

## Rank inversion checks in scikit-criteria

In scikit-criteria we implement these checks via the classes `RankInvariantChecker` and `TransitivityChecker`, which are available in the `skcriteria.ranksrev` module.

The following sections will specify how these classes can be used to test for rank reversals and to analyze the results they yield.

## Test criterion 1 - Rank Invariance

Test criterion 1 evaluates the stability of an MCDM method's top-ranked alternative under minor degradations of non-optimal alternatives, which roughly attempts to detect rank reversals due to irrelevant changes.

### Experiment setup (TODO: make this a propper example)

In [5]:
import skcriteria as skc
from skcriteria.pipeline import mkpipe
from skcriteria.preprocessing.invert_objectives import InvertMinimize
from skcriteria.preprocessing.scalers import SumScaler, VectorScaler
from skcriteria.preprocessing.filters import FilterNonDominated
from skcriteria.agg.simple import WeightedSumModel

method = mkpipe(
    InvertMinimize(),
    FilterNonDominated(),
    SumScaler(target="weights"),
    VectorScaler(target="matrix"),
    WeightedSumModel(),
)

# Load Van 2021 Evaluation Dataset of cryptocurrencies
dm = skc.datasets.load_van2021evaluation(windows_size=7)

In [4]:
from skcriteria.ranksrev import RankInvariantChecker

checker1 = RankInvariantChecker(dmaker=method, repeat=10, random_state=42)
# results1 = checker1.evaluate(dm)

## Test criterions 2 & 3 - Transitivity and Recomposition Consistency

Test criterions 2 and 3 are closely connected in their goal of evaluating the internal consistency and robustness MCDMs via problem decomposition.
The former ensures that transitivity holds across pairwise comparisons, that is, if alternative $A_1 \succ A_2$, and $A_2 \succ A_3$, then $A_1 \succ A_3$; and the latter builds up a total ordering out of these relationships to recompose them into a complete global ranking to test against the original ranking.

Because of this similarity, we opted to unify both criteria into a single class `TransitivityChecker`, that runs both tests secuentially and yields all the relevant details that were encountered in the process.

In the following sections we'll be showing examples using variants of ELECTRE MCDMs because of their strong tendencies to fail these criteria due to how they're implemented.

### Basic experiment

### Untying and Recomposition strategies